### Database

In [1]:
! hive --database stackoverflow_ -e 'show tables'


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.355 seconds
OK
posts
users
Time taken: 0.612 seconds, Fetched: 2 row(s)


### posts

In [2]:
%%writefile describe.hql
USE stackoverflow_;
DESCRIBE posts;

Overwriting describe.hql


In [3]:
! hive -f describe.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.254 seconds
OK
id                  	int                 	                    
post_type_id        	tinyint             	                    
date                	string              	                    
owner_user_id       	int                 	                    
parent_id           	int                 	                    
score               	int                 	                    
favorite_count      	int                 	                    
tags                	array<string>       	                    
year                	string              	                    
month               	string              	                    
	 	 
# Partition Information	 	 
# col_name            	data_type           	comment             
	 	 
year                	string              	                    
month               	string          

In [4]:
%%writefile post.hql
USE stackoverflow_;

SELECT *
FROM 
    posts
WHERE
    tags IS NOT NULL AND
    year IS NOT NULL AND
    month IS NOT NULL
LIMIT 10
;

Overwriting post.hql


In [5]:
! hive -f post.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.12 seconds
OK
4004	1	2008-08-06T21:18:47.357	525	NULL	31	16	[".net","authentication","encryption","ssl"]	2008	2008-08
7237	1	2008-08-10T16:46:12.640	799	NULL	10	2	["sql-server","database"]	2008	2008-08
8472	1	2008-08-12T04:59:35.017	1	NULL	320	323	["security","language-agnostic","captcha"]	2008	2008-08
9303	1	2008-08-12T23:04:10.770	889	NULL	12	1	["c#","regex","perl"]	2008	2008-08
9355	1	2008-08-13T00:35:50.977	234	NULL	15	5	["windows","explorer","windows-shell"]	2008	2008-08
9831	1	2008-08-13T13:49:51.963	1491425	NULL	3	NULL	["javascript","drag-and-drop","mootools"]	2008	2008-08
10260	1	2008-08-13T19:18:07.667	203	NULL	6	NULL	["asp.net"]	2008	2008-08
10810	1	2008-08-14T08:38:24.760	966	NULL	3	1	["c#","winforms","authorization","adam","azman"]	2008	2008-08
12330	1	2008-08-15T15:06:31.873	NULL	NULL	6	1	["vbscript","wmi"]	2008	2008-08
126

### users

In [6]:
%%writefile describe.hql
USE stackoverflow_;
DESCRIBE users;

Overwriting describe.hql


In [7]:
! hive -f describe.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.205 seconds
OK
id                  	int                 	                    
reputation          	int                 	                    
creation_date       	string              	                    
display_name        	string              	                    
location            	string              	                    
views               	int                 	                    
up_votes            	int                 	                    
down_votes          	int                 	                    
age                 	int                 	                    
account_id          	int                 	                    
Time taken: 1.29 seconds, Fetched: 10 row(s)


In [8]:
%%writefile user.hql
USE stackoverflow_;

SELECT *
FROM 
    users
WHERE
    creation_date IS NOT NULL AND
    display_name IS NOT NULL AND
    location IS NOT NULL
LIMIT 10
;

Overwriting user.hql


In [9]:
! hive -f user.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.177 seconds
OK
37	4146	2008-08-01T12:44:00.723	Wally Lawless	Georgetown, Canada	654	256	18	35	28
169	4825	2008-08-02T22:51:03.507	Arnold Zokas	London, United Kingdom	585	1956	14	NULL	138
194	15481	2008-08-03T10:56:49.987	Adam Haile	United States	1973	217	18	33	157
338	37578	2008-08-04T18:34:44.117	Frank Krueger	United States	3941	2411	93	36	272
480	276	2008-08-06T08:40:05.797	mdy	Asia	79	49	0	28	381
756	2358	2008-08-08T15:31:50.013	Simon Gillbee	Pearland, TX	478	352	25	45	587
923	58	2008-08-10T14:25:39.540	AnotherHowie	United Kingdom	32	2	0	45	706
1097	8036	2008-08-12T11:40:09.933	Geoff	Stow, OH	527	473	30	46	838
1462	2011	2008-08-15T17:26:18.470	crono	Germany	119	73	1	35	1101
1539	2476	2008-08-16T13:31:36.823	Ryan P	United States	233	72	9	33	1158
Time taken: 2.382 seconds, Fetched: 10 row(s)


## Hive assignment. Task3
Use Hive to complete the following task. Input tables was described in Hive Task1 and Hive Task2.

Calculate number of questions and answers depending on users' age. Use аge from 'users' table, filter out users if their age is undefined. Output format:
```
age <tab> number of questions <tab> number of answers
```

Note:
    - post_type_id = 1: questions
    - post_type_id = 2: answers

In [10]:
%%writefile query.hql

USE stackoverflow_;

DROP TABLE IF EXISTS posts_question_answer;
CREATE TABLE IF NOT EXISTS
    posts_question_answer AS
SELECT
    owner_user_id,
    IF(post_type_id = 1, 1, 0) AS questions,
    IF(post_type_id = 2, 1, 0) AS answers
FROM 
    posts
;

Overwriting query.hql


In [11]:
%%writefile query.hql -a

SELECT
    age,
    SUM(questions),
    SUM(answers)
FROM 
    posts_question_answer
LEFT JOIN 
    users 
ON
    users.id = posts_question_answer.owner_user_id
WHERE
    age IS NOT NULL
GROUP BY
    age
ORDER BY
    age
;

Appending to query.hql


In [12]:
%%bash
hive -f query.hql 2> out.log
cat out.log >&2

14	1	0
15	1	2
16	2	0
17	0	1
18	4	1
19	1	1
20	0	2
21	11	24
22	6	18
23	12	15
24	16	27
25	20	33
26	23	44
27	28	56
28	24	37
29	24	66
30	26	67
31	17	33
32	13	48
33	11	40
34	24	36
35	12	42
36	8	64
37	9	35
38	6	17
39	3	7
40	1	13
41	5	20
42	5	22
43	2	26
44	7	35
45	1	4
46	7	9
47	1	1
48	1	1
49	1	26
50	1	26
51	4	5
52	0	2
53	0	2
54	0	1
57	0	3
58	1	57
60	0	6
61	0	3
64	2	1
86	0	1
96	3	1



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.209 seconds
OK
Time taken: 0.12 seconds
Query ID = jovyan_20180207150909_f76eaa12-5156-4973-bd7d-b7c28d431f80
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1518008069663_0068, Tracking URL = http://b5f1542b981e:8088/proxy/application_1518008069663_0068/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1518008069663_0068
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2018-02-07 15:09:39,040 Stage-1 map = 0%,  reduce = 0%
2018-02-07 15:10:01,225 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 10.58 sec
MapReduce Total cumulative CPU time: 10 seconds 580 msec
Ended Job = job_1518008069663_0068
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filtered out by condition reso

### Simplify into 1 query

In [13]:
%%writefile query_2.hql

USE stackoverflow_;

SELECT
    age,
    COUNT(questions),
    COUNT(answers)
FROM (
    SELECT
        owner_user_id,
        IF(post_type_id = 1, TRUE, NULL) AS questions,
        IF(post_type_id = 2, TRUE, NULL) AS answers
    FROM 
        posts
) AS posts_question_answer
LEFT JOIN 
    users 
ON
    users.id = posts_question_answer.owner_user_id
WHERE
    age IS NOT NULL
GROUP BY
    age
ORDER BY
    age
;

Overwriting query_2.hql


In [14]:
%%bash
hive -f query_2.hql 2> out.log
cat out.log >&2

14	1	0
15	1	2
16	2	0
17	0	1
18	4	1
19	1	1
20	0	2
21	11	24
22	6	18
23	12	15
24	16	27
25	20	33
26	23	44
27	28	56
28	24	37
29	24	66
30	26	67
31	17	33
32	13	48
33	11	40
34	24	36
35	12	42
36	8	64
37	9	35
38	6	17
39	3	7
40	1	13
41	5	20
42	5	22
43	2	26
44	7	35
45	1	4
46	7	9
47	1	1
48	1	1
49	1	26
50	1	26
51	4	5
52	0	2
53	0	2
54	0	1
57	0	3
58	1	57
60	0	6
61	0	3
64	2	1
86	0	1
96	3	1



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.136 seconds
Query ID = jovyan_20180207151111_a55b0c10-38d6-4360-901a-2717fb196d81
Total jobs = 2
Execution log at: /tmp/jovyan/jovyan_20180207151111_a55b0c10-38d6-4360-901a-2717fb196d81.log
2018-02-07 03:12:08	Starting to launch local task to process map join;	maximum memory = 477626368
2018-02-07 03:12:09	Dump the side-table for tag: 1 with group count: 64208 into file: file:/tmp/jovyan/5c96d2fb-f245-4f45-89f4-ce9ca7ff6a55/hive_2018-02-07_15-11-57_772_5083163349499027109-1/-local-10005/HashTable-Stage-2/MapJoin-mapfile01--.hashtable
2018-02-07 03:12:10	Uploaded 1 File to: file:/tmp/jovyan/5c96d2fb-f245-4f45-89f4-ce9ca7ff6a55/hive_2018-02-07_15-11-57_772_5083163349499027109-1/-local-10005/HashTable-Stage-2/MapJoin-mapfile01--.hashtable (1425472 bytes)
2018-02-07 03:12:10	End of local task; Time Taken: 1.922 sec.
Execution completed succ